<a href="https://colab.research.google.com/github/k1-furuya/rstan001/blob/main/%E9%A0%BB%E5%BA%A6%E8%AB%96AB%E3%83%86%E3%82%B9%E3%83%88t%E6%A4%9C%E5%AE%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

従来のWEBページAと、改善後のページBをランダムに表示するABテストを実施し、その効果を検証する。意図的にサンプルサイズは小さくしている。


In [1]:
import pandas as pd
# 使うデータの視覚化
data = {
    'ページ': ['従来のページ', '改善後のページ'],
    '表示数': [40, 25],
    'クリック数': [2, 2]
}
df = pd.DataFrame(data)

df

,ページ,表示数,クリック数
0,従来のページ,40,2
1,改善後のページ,25,2


In [2]:
import numpy as np
import scipy.stats as stats

In [3]:
# グループAの結果
n_a = 40  # 表示数
obs_a = 2  # ヒット数

# グループBの結果
n_b = 25  # 表示数
obs_b = 2  # ヒット数


In [4]:
# 各グループのコンバージョン率の推定値を計算

p_a = obs_a / n_a
p_b = obs_b / n_b


In [5]:
# コンバージョン率の差
p_diff = p_b - p_a


In [6]:
# 各グループのコンバージョン率の標準誤差（SE）を計算
se_a = np.sqrt(p_a * (1 - p_a) / n_a)
se_b = np.sqrt(p_b * (1 - p_b) / n_b)

In [7]:
# コンバージョン率の差の標準誤差を計算
se_diff = np.sqrt(se_a**2 + se_b**2)

In [8]:
# tスコアを計算
t_score = p_diff / se_diff


In [11]:
# 自由度を計算します。
# ここではウェルチのt検定の自由度の近似式を使用。
df = (se_diff**4) / ((se_a**4 / (n_a - 1)) + (se_b**4 / (n_b - 1)))

In [12]:
# p値を計算
p_value = stats.t.sf(np.abs(t_score), df) * 2  # 両側検定

t_score, p_value, df

(0.46673178317387404, 0.6430499461438455, 42.96447398744266)

In [13]:
# 95%信頼区間を計算
# t分布を使用して、信頼区間の境界を計算
t_critical = stats.t.ppf(1 - 0.05 / 2, df)  # 95%信頼区間のt値
ci_lower = p_diff - t_critical * se_diff  # 信頼区間の下限
ci_upper = p_diff + t_critical * se_diff  # 信頼区間の上限

ci_lower, ci_upper

(-0.09962951043724405, 0.15962951043724405)

グループBのコンバージョン率がグループAよりも最大15.9%ポイント高く、最大9.9%ポイント低い可能性があることを示しています。 信頼区間にゼロが含まれているため、統計的に有意な差があるとは言えません。信頼区間の幅が広い（約25.8%ポイント）ことから、コンバージョン率の差に関する推定にはかなりの不確実性があることがわかります。

p値が0.643という結果は、ABテストで観察されたコンバージョン率の差について、統計的に有意な差がないと解釈できます。頻度論的アプローチでは、p値が一般的に設定される有意水準（ここでは0.05）よりも大きい場合、帰無仮説（ここではグループAとグループBの間にコンバージョン率の差がないという仮説）を棄却するには十分な証拠が得られていないと考えます。

言い換えると、p値0.643は、観察されたコンバージョン率の差が偶然によっても生じうる範囲内であるということを意味しています。そのため、このテストではグループAとグループBの間に**統計的に意味のある差があるとは言えないという結論**に至ります。

**ベイズABテストと比較したときの頻度論ABテストのいくつかの欠点**

1、***事前情報の組み込み***
   過去のデータや専門家の意見といった事前情報を直接組み込むことができません。一方ベイズでは事前分布を通じてこれらの情報をモデルに組み込むことができます。

2、***結果の解釈***
   頻度論では、p値や信頼区間の解釈がややこしく、例えばp値が0.05の時、帰無仮説が真であると仮定した際に、観測された結果と同じかそれ以上に極端な結果が得られる確率が5%であると解釈します。ここでは単に帰無仮説の下での結果の確率を示すだけで、効果の大きさや方向についての情報は含みません。

　一方ベイズでは効果が〇〇～△△の確率は××％であるというように、直観的に理解しやすくビジネスの意思決定に繋げやすいと思われます。

3、***サンプルサイズの問題***
   頻度論ではサンプルサイズが小さいと、実際には存在する差を見つけ出せなかったり（検出力が低い）、不確実性が高くなり、信頼区間が広がって結果の解釈が困難になることがあります。一方ベイズでは事前情報をモデルに組み込むことで、限られたデータからでも情報を引き出すことができます。

4、***結論の出し方***  p値に基づく頻度論的な結論は、しばしば「有意か否か」という二分法的な結果になりがちです。一方ベイズでは、結果の不確実性が確率分布として表現されるため、より豊富な情報が得られます。

***ABテストに限らずベイズ手法の危うさ***

事前分布の選択が結果に影響を与える可能性があり、強い事前分布を使用する場合は、事前の知識がデータよりも結果に大きな影響を及ぼすことがあるので注意する必要があります。